# 目标

    1、复写一遍Tensorflow定义的resnet结构。
    
resnet_152:

    1、一层 7*7卷积
    2：【1x1x64,3x3x64,1x1x256】x3 卷积
    3：【1x1x128,3x3x128,1x1x512】x8 卷积
    4：【1x1x256,3x3x256,1x1x1024】x36 卷积
    5：【1x1x512,3x3x512,1x1x2048】x3 卷积
    6：一层 FC

In [119]:
from datetime import datetime
import math
import time
import collections
import tensorflow as tf
slim = tf.contrib.slim
LOG_DIR='./logs/forward/'

In [6]:
'''
1、class Block(object)中，object表示Block的父类是object。
2、collections.namedtuple()本质上是一个tuple的子类，
   但是可以像类一样操作。比如定义名为Block的块，属性有下面三个。
3、下面的做法就是定义了一个继承自namedtuple的子类，Block
'''
class Block(collections.namedtuple('Block',
                                   ['scope','unit_fn','args'])):
        
    '''
  使用collections.namedtuple设计ResNet基本模块组的name tuple，
  并用它创建Block的类
  只包含数据结构，不包含具体方法。
  
  定义一个典型的Block，需要输入三个参数：
  scope：Block的名称
  unit_fn：ResNet V2中的残差学习单元 
  args：Block的args。
  '''

In [9]:
'''
定义降采样方法

    注意，这里虽然使用了maxpool，
    但是本质上是用了降采样，而不是池化技术来进行尺寸缩减的。
'''
def subsample(inputs,factor,scope=None):
    '''
    inputs:输入tensor
    factor：采样因子（理解为隔多少个像素取一个样本
    scope：命名空间
    '''
    if inputs == 1:
        return inputs
    else:
        return slim.max_pool2d(inputs,[1,1],stride=factor,scope=scope)

In [107]:
'''
卷积层
    目的：
        让输出feature-map的尺寸和输入的一样。
    因此：
        1、当stride=1时，直接padding=’SAME‘
        2、当stride不为1时，需要自己来做padding
'''
def conv2_same(inputs,num_outputs,kernel_size,stride,scope=None):
    if stride == 1:
        return slim.conv2d(inputs,num_outputs,kernel_size,stride=1,
                           padding='SAME',scope=scope)
    else:
        '''
            inputs有4维：batch，h,w,channel,我们要在h和w上padding
            pad中的第一个[0，0],表示我们在batch这一维，开头不补，结尾不补。
            其他同理
            呃，这里其实没有处理吧…感觉没变化的啊
            试验了，是没变化的…不明白为什么要怎么做
        '''
        pad_total = kernel_size-1
        pad_beg = pad_total // 2
        pad_end = pad_total-pad_beg
        inputs = tf.pad(inputs,[[0,0],[pad_beg,pad_end],
                                [pad_beg,pad_end],[0,0]])
        return slim.conv2d(inputs,num_outputs,kernel_size,stride=stride,
                           padding='VALID',scope=scope)

In [108]:
'''
定义堆叠block的函数
只有用@slim.add_arg_scope修饰过的参数才能用 with slim.arg_scope()来设置参数
'''
@slim.add_arg_scope
def stack_blocks_dense(net,blocks,outputs_collections=None):
    '''
  Args:
    net: A `Tensor` of size [batch, height, width, channels].输入。
    blocks: 是之前定义的Block的class的列表。
    outputs_collections: 收集各个end_points的collections。
  Returns:
    net: Output tensor 
    '''
    
    for block in blocks:
        with tf.variable_scope(block.scope,'block',[net]) as sc:
            '''
                1、block.args是形如这样的list
                    [(256, 64, 1), (256, 64, 1), (256, 64, 2)]
                2、unit指的就是里面的每一个(256,64,1)这样的
                3、block.unit_fn在这里指的就是bottleneck这个函数
            '''
            for i ,unit in enumerate(block.args):
                with tf.variable_scope('unit_%d'%(i+1),values=[net]):
                    unit_depth,unit_depth_bottleneck,unit_strid = unit
                    '''
                        虽然这里的block.unit_fn没有定义outputs_collections
                        但是之前用arg_scope定义过了，所以还是有值的。
                    '''
                    net = block.unit_fn(net,
                                        depth=unit_depth,
                                        depth_bottleneck = unit_depth_bottleneck,
                                        stride = unit_strid)
            net = slim.utils.collect_named_outputs(outputs_collections,sc.name,net)
            print(net)
    return net

In [109]:
'''
创建arg_scope,用来定义模型超参数的默认值
'''
def resnet_arg_scope(is_training=True,
                     weight_decay = 0.0001,
                     batch_norm_decay = 0.997,
                     batch_norm_epsilon = 1e-5,
                     batch_norm_scale = True):
    batch_norm_params = {
        'is_training':is_training,
        'decay':batch_norm_decay,
        'epsilon':batch_norm_epsilon,
        'scale':batch_norm_scale,
        'updates_collections':tf.GraphKeys.UPDATE_OPS
    }
    
    with slim.arg_scope([slim.conv2d],
                        weights_regularizer = slim.l2_regularizer(weight_decay),
                        weights_initializer = slim.variance_scaling_initializer(),
                        activation_fn = tf.nn.relu,
                        normalizer_fn = slim.batch_norm,
                        normalizer_params = batch_norm_params):
        with slim.arg_scope([slim.batch_norm],**batch_norm_params):
            with slim.arg_scope([slim.max_pool2d],padding='SAME') as arg_sc:
                return arg_sc

In [114]:
'''
核心：定义一个残差单元的地方

关于tf.variable_scope中的values解释：https://stackoverflow.com/questions/40164583/tensorflows-tensorflow-variable-scope-values-parameter-meaning
大意就是能够是来自不同graph的variable在一起被操作吧

'''
@slim.add_arg_scope
def bottleneck(inputs,depth,depth_bottleneck,stride,
               outputs_collections=None,scope=None):
    '''
    Args:
    inputs: A tensor of size [batch, height, width, channels].
    depth、depth_bottleneck，stride三个参数是前面blocks类中
        depth表示这个残差块最终输出channel
        depth_bottleneck表示这个残差块中间的卷积核输出channel
        stride表示卷积步长
    outputs_collections: 是收集end_points的collection
    scope: 是这个unit的名称。
    '''
    with tf.variable_scope(scope,'bottleneck_v2',[inputs]) as sc:
        depth_in = slim.utils.last_dimension(inputs.get_shape(),
                                             min_rank=4)
        '''
            1、回忆resnet的顺序，
                第一条路：先normal，再激活，然后再卷积
                第二条路：shortcuts
            
        '''
        preact = slim.batch_norm(inputs,activation_fn=tf.nn.relu,scope='preact')
        
        
        '''
            下面是short_cuts部分
        '''
        
        if depth == depth_in:
            shortcut = subsample(inputs,stride,'shortcut')
        else:
            shortcut = slim.conv2d(preact,depth,[1,1],stride=stride,
                                   normalizer_fn=None,activation_fn=None,
                                   scope='shortcut')
            
        '''
            下面是非short_cuts部分
        '''
        residual = slim.conv2d(preact,depth_bottleneck,[1,1],
                               stride=1,scope='conv1')
        
        # 就只有这里stride可能是2，所以需要特殊写一个conv方法，来处理
        residual = conv2_same(residual,depth_bottleneck,3,stride,
                              scope='conv2')
        
        residual = slim.conv2d(residual,depth,[1,1],stride=1,
                               normalizer_fn=None,
                               activation_fn=None,scope='conv3')
        
        output = shortcut+residual
        '''
            这里大致意思是
                    把output这个tensor
                    以别名sc.name的方式
                    加入到名字为outputs_collections中去
                然后再返回output这个tensor
        '''
        return slim.utils.collect_named_outputs(outputs_collections,
                                                sc.name,output)
        
        

In [115]:
'''
定义resnet-v2的主函数
'''
def resnet_v2(inputs,blocks,num_classes=None,
              global_pool=True,include_root_block=True,
              reuse=None,scope=None):
    '''
        之前试着把这个reuse=reuse删了，发现也没什么问题，表明其实这里并没有需要reuse的参数
        
    '''
    with tf.variable_scope(scope,'resnet_v2',[inputs],reuse=reuse) as sc:
        '''
            拿到了一个字符串，作为namescope
        '''
        end_points_collection = sc.original_name_scope + '_end_points'
        
        '''
            bottleneck和stack_blocks_dense
            都是我们刚刚定义的方法
            定义他们的name_scope都是end_points_collection
        '''
        with slim.arg_scope([slim.conv2d,bottleneck,stack_blocks_dense],
                            outputs_collections = end_points_collection):
            net = inputs
            if include_root_block:
                '''
                    这里做的是ResNet中的第一层，7x7的卷积
                '''
                with slim.arg_scope([slim.conv2d],
                                    activation_fn=None,
                                    normalizer_fn=None):
                    net = conv2_same(net,64,7,stride = 2,scope='conv1')
                net = slim.max_pool2d(net,[3,3],stride=2,scope='pool1')
            '''
                进行block的堆叠
            '''
            net = stack_blocks_dense(net,blocks)
            '''
                对堆叠完的block进行BN层操作
            '''
            net = slim.batch_norm(net,activation_fn=tf.nn.relu,scope='postnorm')
            
            '''
                全局平均池化，【1，2】指的是net的维度
            '''
            if global_pool:
                net = tf.reduce_mean(net,[1,2],name='pool5',keep_dims=True)
            '''
                用1*1压缩/放大全局平均池化结果，作为最后的输出
            '''
            if num_classes is not None:
                net = slim.conv2d(net,num_classes,[1,1],activation_fn=None,normalizer_fn=None,scope='logits')
            
            '''
                根据名字end_point_collection将所有变量转换为dict
            '''
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            '''
                并且把最终结果也加进去
            '''
            if num_classes is not None:
                end_points['predictions'] = slim.softmax(net,scope='predictions')
            return net,end_points


In [116]:
def resnet_v2_152(inputs,num_classes=None,
                  global_pool=True,reuse=None,
                  scope='resnet_v2_152'):
    blocks = [
            Block('block1',bottleneck,[(256,64,1)]*2 + [(256,64,2)]),
            Block('block2',bottleneck,[(512,128,1)]*7 + [(512,128,2)]),
            Block('block3',bottleneck,[(1024,256,1)]*35+[(1024,256,2)]),
            Block('block4',bottleneck,[(2048,512,1)]*3)
             ]
    return resnet_v2(inputs,blocks,num_classes,global_pool,
                     include_root_block=True,reuse=reuse,scope=scope)

In [117]:
'''
测试函数
'''
def time_tensorflow_run(session,target,info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    
    for i in range(num_batches + num_steps_burn_in):
        start_time =time.time()
        _ = sess.run(target)
        duration = time.time()-start_time
        if i >= num_steps_burn_in:
            if not i%10:
                print('%s: step %d, duration = %.3f' %
                       (datetime.now(), i - num_steps_burn_in, duration))
            total_duration+=duration
            total_duration_squared+=duration**2
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print ('%s: %s across %d steps, %.3f +/- %.3f sec / batch' %
           (datetime.now(), info_string, num_batches, mn, sd))

In [120]:
tf.reset_default_graph()
batch_size = 32
height,width = 224,224
inputs = tf.random_uniform(shape=(batch_size,height,width,3)
                           ,dtype=tf.float32)

with slim.arg_scope(resnet_arg_scope(is_training=False)):
    net,end_points = resnet_v2_152(inputs,1000)
    
    
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    tf.summary.FileWriter(LOG_DIR,sess.graph)
    num_batches =100
    time_tensorflow_run(sess,net,'forward')

Tensor("resnet_v2_152/block1/unit_3/bottleneck_v2/add:0", shape=(32, 28, 28, 256), dtype=float32)
Tensor("resnet_v2_152/block2/unit_8/bottleneck_v2/add:0", shape=(32, 14, 14, 512), dtype=float32)
Tensor("resnet_v2_152/block3/unit_36/bottleneck_v2/add:0", shape=(32, 7, 7, 1024), dtype=float32)
Tensor("resnet_v2_152/block4/unit_3/bottleneck_v2/add:0", shape=(32, 7, 7, 2048), dtype=float32)
2019-02-09 22:55:34.354135: step 0, duration = 9.683


KeyboardInterrupt: 

In [91]:
for k in end_points.keys():
    print(k,end_points[k])
    break

resnet_v2_152/conv1 Tensor("resnet_v2_152/conv1/BiasAdd:0", shape=(32, 112, 112, 64), dtype=float32)
